In [3]:
import os
import pandas as pd
import numpy as np
import openpyxl

class Binaryclass:
    
    #attributes
    train = ''
    test = ''
    trueclass_train = ''
    format_perfsheet= True   
    
    #create mapping dataframe    
    df_result = pd.DataFrame
    df_prediction = pd.DataFrame 
    df_successrate = pd.DataFrame
        
    #define initialization function
    def __init__(self,train,test,trueclass_train):
        self.train = train
        self.test = test
        self.trueclass_train = trueclass_train
        result_data = {'feature':[],'category':[],'survived':[]}
        self.df_result = pd.DataFrame(result_data)  
    
    #2.	Construct a machine learning model using OneR 
    #for features: gender, pclass, sibsp, parch, #embarked. [50 points; 10 points per feature]    
    def OneR_model(self,col_name,is_test):
        #intialising data
        if is_test:
            df = self.test
            df_column = self.test[col_name]
        else:
            df = self.train
            df_column = self.train[col_name]            
        
        #check NA or null values and forward fill
        if df_column.isnull().values.any():
            df_column = df_column.fillna(method ='ffill') 
        
        #extracting unique category values in the column
        unique_values = np.unique(df_column)
   
        #filter ID column         
        self.df_prediction = df.filter(items = ['ID'])
        self.df_prediction['Prediction'] = np.NaN                
        
        #for each category in the column majority survival value is calculated
        for category in unique_values:
            #calculating majority survival value for a category
            is_category = (df_column == category)
            
            #For test data accessing class from training result dataframe
            if is_test:
                
                classifier = self.df_result['survived'][(self.df_result['category']==category) & 
                                                        (self.df_result['feature'] == col_name)]
                #adding prediction column to the dataframe
                #self.df_prediction['Prediction'][is_category] = int(classifier)
                self.df_prediction.loc[is_category,'Prediction'] = int(classifier)
            
            #For train data creating OneR model classifier
            else:
                ser_survived_count = self.trueclass_train[is_category].value_counts()
                #saving majority survival value as classifier
                if ser_survived_count.size == 2:
                    classifier = ser_survived_count[1] > ser_survived_count[0]
                else:
                    classifier = ser_survived_count.index[0]

                #creating dataframe with ID and prediction                
                self.df_prediction.loc[is_category,'Prediction'] = int(classifier)
                #saving it in a dataframe
                append_data = {'feature':col_name,'category':category,'survived':classifier}
                self.df_result = self.df_result.append(append_data,ignore_index = True)

        #writing excel sheet with given column name
        req_sheetname = col_name+ "_Based_Prediction"
        if is_test:
            filename_dest = "titanic_test_predictions.xlsx"
            self.Write_TestExcel(filename_dest,req_sheetname)
            
        return self.df_result
      
    #4.For each feature, store the passenger IDs and class labels in the appropriate worksheet 
    #(e.g., Gender_Based_Prediction for gender based OneR) of the attached output file (titanic_test_predictions.xlsx). 
    #writing excel sheet with given column name for Test set with Predictions
    def Write_TestExcel(self,filename_dest,sheetname_dest):
        
        #checking for gender_Based_Prediction sheet's name
        if sheetname_dest == 'gender_Based_Prediction':
            sheetname_dest = 'Gender_Based_Prediction'            
        
        #read target file titanic_test_predictions.xslx
        df_source = pd.read_excel(filename_dest, sheet_name=sheetname_dest)
        df_source = df_source.filter(items = ['ID','Ground truth'])
        
        #creating new dataframe with above columns and prediction column
        df_final = pd.merge(df_source, self.df_prediction, on='ID', how='inner')
        
        #calculate prediction success rate
        success_rate = self.Calc_Successrate(df_final)        
        
        #update performance in appropriate sheet
        sheet_performance = 'Prediction_Success_Rate'        
        Feature = sheetname_dest        
        self.Write_Successrate(filename_dest,sheet_performance,success_rate,Feature)        
        
        #Delete existing sheet
        self.Del_Sheet(filename_dest,sheetname_dest)
        
        #if file doesn't exist use write 'w' mode, else use appendmode 'a'
        if not os.path.isfile(filename_dest):
            write_mode = 'w'
        else:
            write_mode= 'a'    
    
        with pd.ExcelWriter(filename_dest,engine='openpyxl',mode=write_mode) as writer:
            df_final.to_excel(writer, sheet_name=sheetname_dest,index=False)
            # Close the Pandas Excel writer and output the Excel file.
            writer.save()
            
    #5.Compute success rate for the test set.
    #Success rate is the total number of correct predictions divide by the total instances. [10 points]
    def Calc_Successrate(self,df_input):
        pred_difference = sum(abs(df_input['Ground truth']-df_input['Prediction']))
        total_rows = df_input['Ground truth'].shape[0]
        error_rate = pred_difference/total_rows
        success_rate = round(1-error_rate,2)

        return success_rate
    
    #updating excel sheet with success rate of each model in Prediction_Success_Rate sheet
    def Write_Successrate(self,filename_dest,sheet_performance,success_rate,Feature):
        df_source = pd.read_excel(filename_dest, sheet_name=sheet_performance)
        
        is_category = (df_source['Feature'] == Feature)
        if self.format_perfsheet:
            df_source.loc[is_category,'Success Rate '] = success_rate
            format = lambda x: (x*100)
            df_source['Success Rate '] = df_source['Success Rate '].map(format)
            self.format_perfsheet = False
        else:
            df_source.loc[is_category,'Success Rate '] = success_rate*100
        
        #assigning result to class' attribute
        self.df_successrate = df_source
        #Delete existing sheet
        self.Del_Sheet(filename_dest,sheet_performance)
        #creating new sheet with updated data
        with pd.ExcelWriter(filename_dest,engine='openpyxl',mode='a') as writer:
            df_source.to_excel(writer, sheet_name=sheet_performance,index=False)
            # Save the Pandas Excel writer and output the Excel file.
            writer.save()
        
    #deleting existing sheet in workbook
    def Del_Sheet(self,filename_dest,sheetname_dest):        
        workbook=openpyxl.load_workbook(filename_dest)        
        del workbook[sheetname_dest]        
        workbook.save(filename_dest)
        workbook.close()1
        
#changing directory to required path
required_path = r"F:\Programming\Python"
os.chdir(required_path)

# 1.	Read-in the training set and test set.
#Reading train and test files as dataframes
train_file_name = "titanic_traning.xlsx"
test_file_name = "titanic_test.xlsx"
df_train = pd.read_excel(train_file_name)
df_test = pd.read_excel(test_file_name)
trueclass_train = df_train['survived']

#creating object of Binary classifier class
Titanic = Binaryclass(df_train,df_test,trueclass_train)
#2.	Construct a machine learning model using OneR for each of the following features: 
#gender, pclass, sibsp, parch, embarked. [50 points; 10 points per feature]
Titanic.OneR_model('gender',is_test = False)
Titanic.OneR_model('pclass',is_test = False)
Titanic.OneR_model('sibsp',is_test = False)
Titanic.OneR_model('parch',is_test = False)
Titanic.OneR_model('embarked',is_test = False)

#OneR model for given training set
print(Titanic.df_result)

#3.For each feature, predict class label (survived or not survived)for every passenger in the test set. [10 points]
#4.For each feature, store the passenger IDs and class labels in the appropriate worksheet 
#(e.g., Gender_Based_Prediction for gender based OneR) of the attached output file (titanic_test_predictions.xlsx). 
Titanic.OneR_model('gender',is_test = True)
Titanic.OneR_model('pclass',is_test = True)
Titanic.OneR_model('sibsp',is_test = True)
Titanic.OneR_model('parch',is_test = True)
Titanic.OneR_model('embarked',is_test = True)

#5.Compute success rate for the test set.
#Success rate is the total number of correct predictions divide by the total instances. [10 points]
print(Titanic.df_successrate)

     feature category  survived
0     gender   female       1.0
1     gender     male       0.0
2     pclass        1       1.0
3     pclass        2       0.0
4     pclass        3       0.0
5      sibsp        0       0.0
6      sibsp        1       1.0
7      sibsp        2       0.0
8      sibsp        3       0.0
9      sibsp        4       0.0
10     sibsp        5       0.0
11     sibsp        8       0.0
12     parch        0       0.0
13     parch        1       1.0
14     parch        2       1.0
15     parch        3       1.0
16     parch        4       0.0
17     parch        5       0.0
18     parch        6       0.0
19     parch        9       0.0
20  embarked        C       1.0
21  embarked        Q       0.0
22  embarked        S       0.0
                     Feature  Success Rate 
0    Gender_Based_Prediction           77.0
1    pclass_Based_Prediction           68.0
2     sibsp_Based_Prediction           61.0
3     parch_Based_Prediction           65.0
4  embarked_